In [1]:
# TODO
# Take subset
# Build models 
# Random Search
# SVM

import numpy as np
import codecs
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score
from scipy.stats import itemfreq

Using TensorFlow backend.


In [2]:
data = pd.read_csv('/Users/krishna/MOOC/IndeedML/dataset/train.tsv', sep='\t')

In [3]:
data['tags']

0       licence-needed supervising-job 5-plus-years-ex...
1        2-4-years-experience-needed salary full-time-job
2                                           part-time-job
3                                          licence-needed
4       5-plus-years-experience-needed full-time-job b...
5         associate-needed 5-plus-years-experience-needed
6       full-time-job ms-or-phd-needed 5-plus-years-ex...
7         bs-degree-needed 5-plus-years-experience-needed
8            bs-degree-needed 2-4-years-experience-needed
9                               full-time-job hourly-wage
10                                         licence-needed
11                            2-4-years-experience-needed
12                                          full-time-job
13                                            hourly-wage
14           bs-degree-needed 2-4-years-experience-needed
15                                            hourly-wage
16           bs-degree-needed 2-4-years-experience-needed
17            

In [67]:
data = pd.read_csv('/Users/krishna/MOOC/IndeedML/dataset/train.tsv', sep='\t')
test = pd.read_csv('/Users/krishna/MOOC/IndeedML/dataset/test.tsv', sep='\t')
data = data.dropna()

In [111]:
data_arry = ['part-time-job',
            'full-time-job',
            'hourly-wage',
            'salary',
            'associate-needed',
            'bs-degree-needed',
            'ms-or-phd-needed',
            "licence-needed",
            '1-year-experience-needed',
            '2-4-years-experience-needed',
            '5-plus-years-experience-needed',
            'supervising-job']

In [68]:
data[1:10]

,tags,description
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,part-time-job,This is a great position for the right person....
3,licence-needed,A large multi-specialty health center is expan...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...
5,associate-needed 5-plus-years-experience-needed,"At Cottage Health System, our facilities are s..."
6,full-time-job ms-or-phd-needed 5-plus-years-ex...,*E Learning Instructional Designer with at lea...
7,bs-degree-needed 5-plus-years-experience-needed,Sales and Service Representative SAF-Hollan...
8,bs-degree-needed 2-4-years-experience-needed,"SAF-Holland, Inc. is currently seeking an Inte..."
9,full-time-job hourly-wage,Immediate opening for full time staffing coord...


In [69]:
def fix_tags(data):
    cat_array = np.zeros([12], dtype=int)
    data_arry = ['part-time-job',
            'full-time-job',
            'hourly-wage',
            'salary',
            'associate-needed',
            'bs-degree-needed',
            'ms-or-phd-needed',
            "licence-needed",
            '1-year-experience-needed',
            '2-4-years-experience-needed',
            '5-plus-years-experience-needed',
            'supervising-job']
     
    split_str = data.split(" ")
    for i in split_str:
        index = data_arry.index(i)
        cat_array[index] = 1
    return cat_array

In [75]:
data['part-time-job'], \
data['full-time-job'], \
data['hourly-wage'],\
data['salary'],\
data['associate-needed'],\
data['bs-degree-needed'], \
data['ms-or-phd-needed'], \
data['licence-needed'], \
data['1-year-experience-needed'], \
data['2-4-years-experience-needed'], \
data['5-plus-years-experience-needed'], \
data['supervising-job']= zip(*data['tags'].map(fix_tags))

In [89]:
data.shape

(3504, 15)

In [76]:
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 1.0,
                             sublinear_tf=True,
                             use_idf=True)

In [101]:
train_vectors = vectorizer.fit_transform(data['description'])
test_vectors = vectorizer.transform(test['description'])
clf = SVC(C=1, kernel='rbf', cache_size=5000,
                  verbose=0)

In [102]:
def eval_models(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average=None)
    return (acc, f1)

In [103]:
models = []
y_hats = np.empty((3504, 12))
for k,v in enumerate(data_arry):
    target = data[v]
    model = clf.fit(train_vectors, target)
    models.append(model)
    print(eval_models(target, model.predict(train_vectors)))
    y_hat = model.predict(train_vectors)
    y_hats[:,k] = y_hat
    print("Observed", itemfreq(target).astype(int))
    print("Predict", itemfreq(y_hat).astype(int))
    print("####")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1118: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.90639269406392697, array([ 0.9508982,  0.       ]))
Observed [[   0 3176]
 [   1  328]]
Predict [[   0 3504]]
####
(0.74743150684931503, array([ 0.85546301,  0.        ]))
Observed [[   0 2619]
 [   1  885]]
Predict [[   0 3504]]
####
(0.87128995433789957, array([ 0.93121855,  0.        ]))
Observed [[   0 3053]
 [   1  451]]
Predict [[   0 3504]]
####
(0.80907534246575341, array([ 0.89446285,  0.        ]))
Observed [[   0 2835]
 [   1  669]]
Predict [[   0 3504]]
####
(0.94035388127853881, array([ 0.96926019,  0.        ]))
Observed [[   0 3295]
 [   1  209]]
Predict [[   0 3504]]
####
(0.72317351598173518, array([ 0.83935078,  0.        ]))
Observed [[   0 2534]
 [   1  970]]
Predict [[   0 3504]]
####
(0.97631278538812782, array([ 0.98801444,  0.        ]))
Observed [[   0 3421]
 [   1   83]]
Predict [[   0 3504]]
####
(0.8504566210045662, array([ 0.91918569,  0.        ]))
Observed [[   0 2980]
 [   1  524]]
Predict [[   0 3504]]
####
(0.90553652968036524, array([ 0.95042684,  

In [104]:
y_hats

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [112]:
for i in y_hats:
    labels = []
    for k,v in enumerate(i):
        if v == 0:
            labels.append(data_arry[k])
    print(labels)
    break

['part-time-job', 'full-time-job', 'hourly-wage', 'salary', 'associate-needed', 'bs-degree-needed', 'ms-or-phd-needed', 'licence-needed', '1-year-experience-needed', '2-4-years-experience-needed', '5-plus-years-experience-needed', 'supervising-job']
